In [1]:
import numpy as np
import sklearn
import underthesea
import json
import os
from tqdm.autonotebook import tqdm
from glob import glob
from pprint import pprint
import nltk
from underthesea import word_tokenize
from sklearn.decomposition import TruncatedSVD
import math
import sklearn


/tmp/ipykernel_1083/2933293930.py:6: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


## Load data in json file

In dataset, item format is json with many fields and coresponding content 

In [4]:
json_fns = glob("./dataset/*.json")
items = []

for fn in json_fns:
    with open(fn, "r") as f:
        items += json.load(f)
print(len(items))
test_item = items[0]
pprint(test_item)

3459
{'Chất liệu': 'chất liệu cotton',
 'Danh Mục': 'danh mục thời trang nữ áo áo thun',
 'Gửi từ': 'gửi từ quận 4 tp hồ chí minh',
 'Kho hàng': 'kho hàng 6',
 'Kiểu tay': 'kiểu tay ngắn tay',
 'Thương hiệu': 'thương hiệu no brand',
 'Xuất xứ': 'xuất xứ quảng châu',
 'desc_2': 'áo thun croptop mịn đẹp  chất thun cotton co giãn thấm hút mồ hôi  '
           'phối quần jean  quần short  chân váy đều xinh  trẻ trung năng động '
           '4 màu trắng đỏ đen xanh aothunfreesize aothuncotton aothunformrong '
           'aophong aothun aodangrong aoniphoisomi aokorea aolechvai aocoduc '
           'aotrevai hoodie hoodie aothunnu',
 'price': '86000',
 'title': 'áo thun cotton croptop ulzzang in số'}


In [4]:
import pandas as pd
df = pd.read_csv('./tdgd_laptop/laptop_all.csv')

In [6]:
items = []

for index, row in df.iterrows():
    items.append(row['full_name'])

### Tokenizer: word segmentation

In [7]:
class Tfidf:
    def __init__(self, title_string = "hello my friend"):
        self.corpus = {}
        self.corpus_counter = {}
        self.object_title = title_string
        self.corpus_len = 0
        
    '''
    Xây dựng TF-IDF cho danh sách documents nhận đuợc
    documents: list<string>
    '''
    def fit_data(self, documents):
#         print(type(documents))
#         if type(documents) != "list":
#             raise ValueError('documents must be type of list')
        
#         đếm số lần xuất hiện của mỗi từ
        for d in documents:
            tokens = word_tokenize(d, format='text').split()
            self.corpus_len += 1
            for t in tokens:
                if t in self.corpus_counter.keys():
                    self.corpus_counter[t] += 1
                else:
                    self.corpus_counter[t] = 1
        
        
        
    def print_data(self):
        print(self.corpus_counter)
        print(len(self.corpus_counter))
        
    def get_tfidf(self, string):
        doc_counter = {}
        k = 2
        tokens = word_tokenize(string, format='text').split()
        for t in tokens:
            if t in doc_counter.keys():
                doc_counter[t] += 1
            else:
                doc_counter[t] = 1
        
        vector_len = len(self.corpus_counter)
        tfidf_vector = np.zeros((vector_len,))
        for i, key in enumerate(self.corpus_counter.keys()):
            if key in doc_counter.keys():
                tf = (k+1)*doc_counter[key]/(k+doc_counter[key])
                idf = math.log((self.corpus_len+1)/(self.corpus_counter[key]))
                tfidf_vector[i] = tf*idf
        return tfidf_vector

In [11]:
class Storage:
    def __init__(self):
        self.tfidf_space = []
        self.tfidf = Tfidf()
        self.svd = TruncatedSVD(n_components=256)
        self.items = []
    
    '''
    items: list<string>
    '''
    def fit_data(self, items):
        self.tfidf.fit_data(items)
        
        for i in items:
            self.tfidf_space.append(self.tfidf.get_tfidf(i))
            self.items.append(i)
            
#         self.svd.fit(self.tfidf_space)
#         self.svd_tfidf_vector = self.svd.transform(self.tfidf_space)
        
    '''
    item: string
    '''
    def get_similiar_items(self, item):
        query_vector = self.tfidf.get_tfidf(item)
        query_vector = np.reshape(query_vector, (1,-1))
        # search
        sim_maxtrix = sklearn.metrics.pairwise.cosine_similarity(query_vector, self.tfidf_space)
        sim_maxtrix = np.reshape(sim_maxtrix, (-1,))
        idx = (-sim_maxtrix).argsort()[:20]
        for _id in idx:
            print(_id, sim_maxtrix[_id])
#             print(newItems[_id]['name'].upper())
            print(self.items[_id], "\n\n")


In [12]:
s = Storage()

item_descriptions = []
for i in items:
    item_descriptions.append(i)
    
s.fit_data(item_descriptions)

In [14]:
s.get_similiar_items("Laptop  Acer  Ryzen 5")

13 0.3217422395559551
 Laptop  MSI  Ryzen 5 5500U 2.1GHz 8 GB 512 GB SSD NVMe PCIe (Có thể tháo ra, lắp thanh khác tối đa 2TB) Card tích hợp Radeon 15.6" Full HD (1920 x 1080) 


101 0.31729649927375625
 Laptop  HP  Ryzen 5 5600H 3.3GHz 8 GB 512 GB SSD NVMe PCIe (Có thể tháo ra, lắp thanh khác tối đa 1TB) Card rời RTX 3050 4GB 16.1" Full HD (1920 x 1080) 


0 0.3126403752194007
 Laptop  Acer  Ryzen 5 5600H 3.3GHz 8 GB 512 GB SSD NVMe PCIe (Có thể tháo ra, lắp thanh khác tối đa 1TB) Card rời GTX 1650 4GB 15.6" Full HD (1920 x 1080) 


123 0.31115383457537266
 Laptop  Acer  Ryzen 5 5500U 2.1GHz 8 GB 512 GB SSD NVMe PCIe (Có thể tháo ra, lắp thanh khác tối đa 1TB) Card rời GTX 1650 4GB 15.6" Full HD (1920 x 1080) 


142 0.3049549057895718
 Laptop  Acer  Ryzen 5 5500U 2.1GHz 8 GB 256 GB SSD NVMe PCIe (Có thể tháo ra, lắp thanh khác tối đa 1TB) Card rời GTX 1650 4GB 15.6" Full HD (1920 x 1080) 


87 0.3049405607303137
 Laptop  Asus  Ryzen 5 5600H 3.3GHz 8 GB 512 GB SSD NVMe PCIe (Có thể thá